# UR5 Simulation forward kinematics
**Author** Eduardo Gomez

Using the Denavit-Hartenberg parameters of the UR5, program the forward kinematics

In [1]:
# Import libraries
import zmqRemoteApi
import numpy as np
from time import sleep
from pathlib import Path
import pandas as pd
import utils

## Setup Data

Here we will set up the data for the Denavit-Hartenberg parameters. The data was from: ...

In [2]:
# First one I think is wrong



The matriz transformations can be expressed like these: 

$$
H_i^{i-1} = Rot_z(\theta_i)T(0,0,d_i)T(a_i, 0, 0)Rotx(\alpha_i)
$$

Esto se lee como una rotacion $\theta$ en $z$, una translacion $d$ en $z$, translacion $a$ en $x$ y rotacion $\alpha$ en $x$,

Here we'll define some utility functions

# Transformation matrix calcs

Because for the moment, all $\theta = 0$, the final coord is in the final matrix

## Setup simulation

To set up the simulation, open the `./sim/ur5-simulation.ttt` file. Then run the following code: 

> If there is an error, either the file is not opened or another script is controlling it

In [3]:
# Create a client to get connected to the zmqremoteApi sernver from CoppeliaSim
client = zmqRemoteApi.RemoteAPIClient()

# Get the remote object 'sim'
sim = client.getObject('sim')

print(sim)

<class 'zmqRemoteApi.sim'>


Here, we set up the id for controlling the robot

In [4]:
UR5_sim_obj_id = "/UR5"
base_id = sim.getObject(UR5_sim_obj_id)

# the last joint is the rotor for the end effector
# does not affect the final position
no_joints = 6
joint_sim_names = [None for _ in range(6)]

joint_sim_names[0] = f"{UR5_sim_obj_id}/joint"

for i in range(1,len(joint_sim_names)):
    link_joint_str = "/link/joint" * i
    joint_sim_names[i] = f"{UR5_sim_obj_id}{link_joint_str}"

connection_name = f"{UR5_sim_obj_id}/connection"

In [5]:
# Assert that names have id. Error is thrown if id doesnt exists
joint_id = [
    sim.getObject(name) for name in joint_sim_names
]
connection_id = sim.getObject(connection_name)

# Calculating Direct kinematics

In [20]:

# Get the relative position in relation from the first jjoint to the second
# Link for doc https://www.coppeliarobotics.com/helpFiles/en/regularApi/simGetObjectMatrix.htm
# For more info in how coppelia treats this stuff https://www.coppeliarobotics.com/helpFiles/en/positionOrientationTransformation.htm
# In the getObjectMatrix the las row [0,0,0,1] was ommited
transformation_matrix_flat = sim.getObjectMatrix(joint_id[1], joint_id[0])
transformation_matrix_incomplete  = np.round(np.array(transformation_matrix_flat).reshape([3,4]), 5)
transformation_matrix_complete  = np.block([[transformation_matrix_incomplete], [np.array([0,0,0,1])]])
print(transformation_matrix_complete)


position_1 = np.round(sim.getObjectPosition(joint_id[1], sim.handle_world), 4)
position_0 = np.round(sim.getObjectPosition(joint_id[0], sim.handle_world), 4)


position_0 = utils.get_homogenous_point_3d(position_0)
position_1_calculated = transformation_matrix_complete @ position_0

utils.print_rounded_np(position_1_calculated)
utils.print_rounded_np(position_1)



[[-0.      -0.      -1.      -0.0703 ]
 [ 0.       1.      -0.       0.     ]
 [ 1.       0.      -0.       0.06605]
 [ 0.       0.       0.       1.     ]]
[-0.0703  0.      0.066   1.    ]
[-0.0703 -0.      0.0661]


In [21]:
def get_transformation_matrix_between_joins(id_prev : str, id_next : str):
    """
    Returns the transformation matrix between each joint
    """
    transformation_matrix_flat = sim.getObjectMatrix(id_next, id_prev)
    transformation_matrix_incomplete  = np.round(np.array(transformation_matrix_flat).reshape([3,4]), 5)
    transformation_matrix_complete  = np.block([[transformation_matrix_incomplete], [np.array([0,0,0,1])]])
    return transformation_matrix_complete


In [36]:
from typing import TypeVar

T = TypeVar("T")

WINDOW_SIZE = 2

def window_iter(array : list[T], window_size : int) -> T:
    for idx in range(len(array) - window_size + 1):
        yield array[idx:idx+window_size]


def get_transformation_matrices_for(join_id_list : list[str]) -> list[np.ndarray]:
    
    h_mats = [
        get_transformation_matrix_between_joins(prev_id, next_id)
        for prev_id, next_id in window_iter(join_id_list, 2)
    ]
        
    return h_mats
        

matrices = get_transformation_matrices_for(joint_id)

assert len(matrices) + 1 == len(joint_id)
print(matrices[0])


[17, 20, 23, 26, 29, 31]
[[-0.      -0.      -1.      -0.0703 ]
 [ 0.       1.      -0.       0.     ]
 [ 1.       0.      -0.       0.06605]
 [ 0.       0.       0.       1.     ]]


Here we stop the simulation

In [ ]:
sim.stopSimulation()